# Stress Level Prediction Pipeline

End-to-end preprocessing and modeling for stress vs non-stress, including aerobic/anaerobic as active negatives.


In [1]:
from __future__ import annotations

import math
import os
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from scipy.signal import welch
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier

np.random.seed(42)


## Config
- Window length 60s, step 30s
- Minimum label coverage 0.7
- Protocol tag mapping per provided notebook
- Data constraints applied during load

In [2]:
# Paths
DEFAULT_DATASET_ROOT = Path("./Datasets")
DATASET_ROOT = Path(os.getenv("DATASET_ROOT", DEFAULT_DATASET_ROOT))

# Windowing
WINDOW_SECONDS = 60
WINDOW_STEP_SECONDS = 30
MIN_LABEL_COVERAGE = 0.7

# Sampling / weighting
PHASE_BALANCE_NO_STRESS_RATIO = 2.0
PHASE_WEIGHT_MAP = {
    "Stroop": 1.5,
    "Opposite Opinion": 1.4,
    "Real Opinion": 1.2,
    "Subtract": 1.2,
    "TMCT": 1.3,
}
EMA_SPAN = 5
PHASE_Z_MIN_STD = 1e-3

# Data constraints
DUPLICATE_CUTS = {"S02": {"ACC": 49545, "BVP": 99091, "EDA": 6195, "TEMP": 6195}}
MISSING_SENSORS = {"f07": {"BVP", "TEMP", "HR", "IBI"}}

STATES = ["STRESS", "AEROBIC", "ANAEROBIC"]

# Stress protocol metadata
STRESS_STAGE_ORDER_S = ["Stroop", "TMCT", "Real Opinion", "Opposite Opinion", "Subtract"]
STRESS_STAGE_ORDER_F = ["TMCT", "Real Opinion", "Opposite Opinion", "Subtract"]
STRESS_TAG_PAIRS_S = [(3, 4), (5, 6), (7, 8), (9, 10), (11, 12)]
STRESS_TAG_PAIRS_F = [(2, 3), (4, 5), (6, 7), (8, 9)]
STRESS_LEVEL_FILES = ["Stress_Level_v1.csv", "Stress_Level_v2.csv"]

def load_stress_levels() -> Dict[str, Dict[str, float]]:
    tables = []
    for fname in STRESS_LEVEL_FILES:
        path = Path(fname)
        if not path.exists():
            continue
        df = pd.read_csv(path, index_col=0)
        df.columns = [str(c).strip() for c in df.columns]
        tables.append(df)
    levels: Dict[str, Dict[str, float]] = {}
    for table in tables:
        for subject, row in table.iterrows():
            subj = str(subject).strip()
            levels[subj] = {col: (float(row[col]) if not pd.isna(row[col]) else np.nan) for col in table.columns}
    return levels


STRESS_LEVELS = load_stress_levels()


STRESS_PHASES = {"Stroop", "TMCT", "Real Opinion", "Opposite Opinion", "Subtract"}
STRESS_LEVEL_BOUNDS = {"low": 3.0, "moderate": 6.0}
TEMPORAL_FEATURES = ["eda_mean", "eda_std", "acc_energy", "temp_mean", "hr_mean"]
FEATURE_EXCLUDE_COLS = {"subject", "state", "phase", "stress_stage", "stress_level", "label", "is_stress", "win_start", "win_end"}

ACC_ACTIVITY_WINDOW_SEC = 2.0
ACC_ACTIVITY_STEP_SEC = 0.5

## Helpers for Empatica format and tags

In [3]:
# Get base subject ID without session suffix.
def base_subject_id(subject: str) -> str:
    return subject.split("_")[0]


# Read signal CSV and return (fs, data, start_timestamp).
def read_signal(path: Path) -> Tuple[float, np.ndarray, pd.Timestamp]:
    with open(path, "r") as f:
        start_line = f.readline()
        if not start_line:
            raise ValueError(f"Missing start timestamp in {path}")
        start = pd.to_datetime(start_line.split(",")[0])
        fs_line = f.readline()
        if not fs_line:
            raise ValueError(f"Missing sample rate in {path}")
        fs = float(fs_line.split(",")[0])
        data = np.genfromtxt(f, delimiter=",")
    if np.isscalar(data):
        if math.isnan(float(data)):
            data = np.empty((0, 1))
        else:
            data = np.array([[float(data)]])
    elif data.size == 0:
        data = np.empty((0, 1))
    else:
        data = np.asarray(data, dtype=float)
        if np.isnan(data).all():
            data = np.empty((0, 1))
        elif data.ndim == 1:
            data = data[:, None]
    return fs, data, start


# Read IBI CSV and return inter-beat intervals in seconds.
def read_ibi(path: Path) -> np.ndarray:
    if not path.exists():
        return np.array([])
    try:
        df = pd.read_csv(path, header=None, skiprows=1)
    except (pd.errors.EmptyDataError, FileNotFoundError):
        return np.array([])
    if df.empty:
        return np.array([])
    df = df.apply(pd.to_numeric, errors="coerce").dropna()
    if df.empty:
        return np.array([])
    col = 1 if df.shape[1] > 1 else 0
    return df.iloc[:, col].to_numpy(dtype=float)


# Read tags.csv and return list of tag timestamps (seconds since start).
def read_tags(path: Path, start_ts: pd.Timestamp) -> List[Tuple[float, float]]:
    try:
        df = pd.read_csv(path, header=None)
    except pd.errors.EmptyDataError:
        return []
    tags = []
    for ts_str in df[0].astype(str):
        ts = pd.to_datetime(ts_str)
        tags.append((ts - start_ts).total_seconds())
    return [(t, t) for t in tags]


# Extract stress intervals from tags based on subject version.
def stress_intervals_from_tags(tags: List[Tuple[float, float]], subject: str) -> List[dict]:
    if not tags:
        return []
    t = [x[0] for x in tags]
    if subject.startswith("S"):
        idx_pairs = STRESS_TAG_PAIRS_S
        stage_order = STRESS_STAGE_ORDER_S
    else:
        idx_pairs = STRESS_TAG_PAIRS_F
        stage_order = STRESS_STAGE_ORDER_F
    base_id = base_subject_id(subject)
    spans = []
    for stage, (i, j) in zip(stage_order, idx_pairs):
        if i < len(t) and j < len(t) and t[j] > t[i]:
            level = STRESS_LEVELS.get(base_id, {}).get(stage)
            spans.append({
                "start": t[i],
                "end": t[j],
                "stage": stage,
                "stress_level": level,
            })
    return spans


# Extract active intervals from tags.
def active_intervals_from_tags(tags: List[Tuple[float, float]], label: str) -> List[dict]:
    if len(tags) < 2:
        return []
    t = [x[0] for x in tags]
    spans = []
    for a, b in zip(t[:-1], t[1:]):
        if b > a:
            spans.append({
                "start": a,
                "end": b,
                "stage": label,
                "stress_level": 0.0,
            })
    return spans




# Map numeric stress score to four-level category.
def stress_bucket(level: float | None, phase: str | None) -> str:
    if phase in {"aerobic", "anaerobic", "rest"}:
        return "no_stress"
    if level is None or pd.isna(level) or level <= 0:
        return "no_stress"
    if level <= STRESS_LEVEL_BOUNDS["low"]:
        return "low_stress"
    if level <= STRESS_LEVEL_BOUNDS["moderate"]:
        return "moderate_stress"
    return "high_stress"


## Resampling and preprocessing

In [4]:
# Resample signal to target frequency.
def resample_to_rate(signal: np.ndarray, src_fs: float, tgt_fs: float) -> np.ndarray:
    if signal.ndim == 1:
        signal = signal[:, None]
    src_len = signal.shape[0]
    duration = src_len / src_fs
    tgt_len = int(duration * tgt_fs)
    src_t = np.linspace(0, duration, src_len)
    tgt_t = np.linspace(0, duration, tgt_len)
    resampled = np.stack([np.interp(tgt_t, src_t, signal[:, i]) for i in range(signal.shape[1])], axis=1)
    if resampled.shape[1] == 1:
        return resampled[:, 0]
    return resampled

# Simple Hampel filter for outlier suppression.
def hampel_filter(x: np.ndarray, k: int = 5, t0: float = 3.0) -> np.ndarray:
    x = x.copy()
    n = len(x)
    for i in range(n):
        window = x[max(i - k, 0): min(i + k, n)]
        med = np.median(window)
        mad = np.median(np.abs(window - med)) or 1e-6
        if abs(x[i] - med) > t0 * mad:
            x[i] = med
    return x

In [5]:
# Approximate ACC activity envelope using sliding RMS to capture movement intensity.
def acc_activity_signal(
    acc_raw: np.ndarray,
    fs: float,
    win_sec: float = ACC_ACTIVITY_WINDOW_SEC,
    step_sec: float = ACC_ACTIVITY_STEP_SEC,
) -> np.ndarray:
    if acc_raw.size == 0 or fs <= 0:
        return np.array([])
    if acc_raw.ndim == 1:
        acc_raw = acc_raw[:, None]
    magnitude = np.linalg.norm(acc_raw, axis=1)
    win = max(1, int(round(win_sec * fs)))
    step = max(1, int(round(step_sec * fs)))
    if len(magnitude) < win:
        rms = float(np.sqrt(np.mean(magnitude ** 2))) if len(magnitude) else 0.0
        return np.array([rms])
    activity = []
    start = 0
    while start + win <= len(magnitude):
        segment = magnitude[start:start + win]
        rms = float(np.sqrt(np.mean(segment ** 2)))
        activity.append(rms)
        start += step
    if start < len(magnitude):
        segment = magnitude[-win:]
        rms = float(np.sqrt(np.mean(segment ** 2)))
        if not activity or abs(activity[-1] - rms) > 1e-9:
            activity.append(rms)
    return np.asarray(activity)

## Feature extraction

In [6]:
def spectral_centroid(signal: np.ndarray, fs: float) -> float:
    if signal.size == 0 or fs <= 0:
        return np.nan
    freqs, psd = welch(signal, fs=fs, nperseg=min(256, len(signal)))
    total = psd.sum()
    if total <= 0:
        return np.nan
    return float(np.sum(freqs * psd) / total)


def bandpower(signal: np.ndarray, fs: float, fmin: float, fmax: float) -> float:
    if signal.size == 0 or fs <= 0 or fmax <= fmin:
        return np.nan
    freqs, psd = welch(signal, fs=fs, nperseg=min(256, len(signal)))
    mask = (freqs >= fmin) & (freqs <= fmax)
    if not np.any(mask):
        return np.nan
    trap = getattr(np, "trapezoid", np.trapz)
    return float(trap(psd[mask], freqs[mask]))


def linear_slope(signal: np.ndarray, fs: float) -> float:
    if signal.size < 2 or fs <= 0:
        return 0.0
    t = np.arange(signal.size) / fs
    slope, _ = np.polyfit(t, signal, 1)
    return float(slope)


def peak_rate(signal: np.ndarray, fs: float) -> float:
    if signal.size < 3 or fs <= 0:
        return 0.0
    duration = signal.size / fs
    if duration <= 0:
        return 0.0
    peaks = np.sum((signal[1:-1] > signal[:-2]) & (signal[1:-1] > signal[2:]))
    return float(peaks / duration)


def eda_features(eda: np.ndarray, fs: float) -> Dict[str, float]:
    if eda.size == 0 or fs <= 0:
        return {
            "eda_mean": np.nan,
            "eda_std": np.nan,
            "eda_slope": np.nan,
            "eda_peak_rate": np.nan,
            "eda_range": np.nan,
            "eda_power_slow": np.nan,
            "eda_power_mid": np.nan,
            "eda_power_fast": np.nan,
            "eda_centroid": np.nan,
        }
    clean = hampel_filter(eda)
    return {
        "eda_mean": float(np.mean(clean)),
        "eda_std": float(np.std(clean)),
        "eda_slope": linear_slope(clean, fs),
        "eda_peak_rate": peak_rate(clean, fs),
        "eda_range": float(np.max(clean) - np.min(clean)),
        "eda_power_slow": bandpower(clean, fs, 0.01, 0.05),
        "eda_power_mid": bandpower(clean, fs, 0.045, 0.25),
        "eda_power_fast": bandpower(clean, fs, 0.25, 1.5),
        "eda_centroid": spectral_centroid(clean, fs),
    }


def temp_features(temp: np.ndarray, fs: float) -> Dict[str, float]:
    if temp.size == 0 or fs <= 0:
        return {
            "temp_mean": np.nan,
            "temp_std": np.nan,
            "temp_slope": np.nan,
            "temp_min": np.nan,
            "temp_max": np.nan,
        }
    return {
        "temp_mean": float(np.mean(temp)),
        "temp_std": float(np.std(temp)),
        "temp_slope": linear_slope(temp, fs),
        "temp_min": float(np.min(temp)),
        "temp_max": float(np.max(temp)),
    }


def hrv_features(ibi: np.ndarray) -> Dict[str, float]:
    if ibi.size < 2:
        return {
            "hr_mean": np.nan,
            "rmssd": np.nan,
            "sdnn": np.nan,
            "pnn50": np.nan,
            "lf_hf": np.nan,
            "sd1": np.nan,
            "sd2": np.nan,
        }
    rr = ibi.astype(float)
    diff = np.diff(rr)
    rmssd = float(np.sqrt(np.mean(diff ** 2))) if diff.size else np.nan
    sdnn = float(np.std(rr))
    pnn50 = float(np.mean(np.abs(diff) > 0.05)) if diff.size else np.nan
    hr_mean = float(60.0 / np.mean(rr)) if np.mean(rr) > 0 else np.nan

    if diff.size:
        sd1 = float(np.sqrt(0.5) * np.std(diff))
    else:
        sd1 = np.nan
    if not np.isnan(sdnn) and not np.isnan(sd1):
        sd2_sq = max(0.0, 2 * (sdnn ** 2) - 0.5 * (sd1 ** 2))
        sd2 = float(np.sqrt(sd2_sq))
    else:
        sd2 = np.nan

    lf_hf = np.nan
    try:
        t = np.cumsum(rr)
        t = t - t[0]
        if t[-1] > 0 and rr.size >= 4:
            fs_interp = 4.0
            grid = np.arange(0, t[-1], 1 / fs_interp)
            if grid.size >= 8:
                interp_rr = np.interp(grid, t[: len(grid)], rr[: len(grid)])
                lf = bandpower(interp_rr, fs_interp, 0.04, 0.15)
                hf = bandpower(interp_rr, fs_interp, 0.15, 0.4)
                if hf and hf > 0:
                    lf_hf = float(lf / hf)
    except Exception:
        lf_hf = np.nan

    return {
        "hr_mean": hr_mean,
        "rmssd": rmssd,
        "sdnn": sdnn,
        "pnn50": pnn50,
        "lf_hf": lf_hf,
        "sd1": sd1,
        "sd2": sd2,
    }


def acc_features(acc_mag: np.ndarray, fs: float, acc_activity: np.ndarray | None = None) -> Dict[str, float]:
    if len(acc_mag) == 0:
        base = {
            "acc_mean": np.nan,
            "acc_std": np.nan,
            "acc_energy": np.nan,
            "acc_peak_freq": np.nan,
            "acc_bandpower_low": np.nan,
            "acc_bandpower_mid": np.nan,
            "acc_bandpower_high": np.nan,
            "acc_mad": np.nan,
        }
    else:
        energy = np.mean(acc_mag ** 2)
        base = {
            "acc_mean": float(np.mean(acc_mag)),
            "acc_std": float(np.std(acc_mag)),
            "acc_energy": float(energy),
            "acc_peak_freq": spectral_centroid(acc_mag, fs),
            "acc_bandpower_low": bandpower(acc_mag, fs, 0.1, 0.5),
            "acc_bandpower_mid": bandpower(acc_mag, fs, 0.5, 2.0),
            "acc_bandpower_high": bandpower(acc_mag, fs, 2.0, 8.0),
            "acc_mad": float(np.median(np.abs(acc_mag - np.median(acc_mag)))),
        }
    if acc_activity is not None and len(acc_activity):
        base.update(
            {
                "acc_activity_mean": float(np.mean(acc_activity)),
                "acc_activity_std": float(np.std(acc_activity)),
                "acc_activity_max": float(np.max(acc_activity)),
            }
        )
    else:
        base.update(
            {
                "acc_activity_mean": np.nan,
                "acc_activity_std": np.nan,
                "acc_activity_max": np.nan,
            }
        )
    return base


def combine_features(eda, eda_fs, acc_mag, acc_fs, temp, temp_fs, ibi, acc_activity=None) -> Dict[str, float]:
    feats = {}
    feats.update(eda_features(eda, eda_fs))
    feats.update(acc_features(acc_mag, acc_fs, acc_activity))
    feats.update(temp_features(temp, temp_fs))
    feats.update(hrv_features(ibi))
    return feats

## Windowing and label assignment

In [7]:
@dataclass
class Window:
    start: float
    end: float
    label: str
    subject: str
    state: str


def window_intervals(duration: float, win_s: int = WINDOW_SECONDS, step_s: int = WINDOW_STEP_SECONDS) -> List[Tuple[float, float]]:
    windows = []
    t = 0.0
    while t + win_s <= duration:
        windows.append((t, t + win_s))
        t += step_s
    return windows


def _span_bounds(span) -> Tuple[float, float]:
    if isinstance(span, dict):
        return span["start"], span["end"]
    return span


def assign_label(win: Tuple[float, float], intervals: Dict[str, List[dict]]) -> Tuple[str | None, dict | None]:
    start, end = win
    length = end - start
    best_label = None
    best_cov = 0.0
    best_span = None
    for lbl, spans in intervals.items():
        label_overlap = 0.0
        label_best_span = None
        label_best_overlap = 0.0
        for span in spans:
            a, b = _span_bounds(span)
            inter = max(0.0, min(end, b) - max(start, a))
            if inter > 0:
                label_overlap += inter
                if inter > label_best_overlap:
                    label_best_overlap = inter
                    label_best_span = span
        coverage = label_overlap / length
        if coverage > best_cov:
            best_cov = coverage
            best_label = lbl
            best_span = label_best_span
    if best_cov >= MIN_LABEL_COVERAGE and best_label is not None:
        return best_label, best_span
    return None, None


def make_label_intervals(state: str, subject: str, tags: List[Tuple[float, float]], duration: float) -> Dict[str, List[dict]]:
    rest_span = {"start": 0.0, "end": duration, "stage": "rest", "stress_level": 0.0}
    if state == "STRESS":
        stress_spans = stress_intervals_from_tags(tags, subject)
        if not stress_spans:
            return {"rest": [rest_span]}
        return {
            "stress": stress_spans,
            "rest": [rest_span],
        }
    else:
        lbl = "aerobic" if state == "AEROBIC" else "anaerobic"
        active = active_intervals_from_tags(tags, lbl)
        return {
            lbl: active,
            "rest": [rest_span],
        }


## Data ingestion: read signals per subject/state

In [8]:
def load_subject_state(state: str, subject: str) -> dict:
    folder = DATASET_ROOT / state / subject
    base_id = base_subject_id(subject)
    sensors = {}
    fs_map = {}
    missing = MISSING_SENSORS.get(base_id, set())

    if not folder.exists():
        raise FileNotFoundError(folder)
    fs_eda, eda_raw, start_ts = read_signal(folder / "EDA.csv")
    sensors["EDA"] = np.squeeze(eda_raw)
    fs_map["EDA"] = fs_eda

    temp_path = folder / "TEMP.csv"
    if "TEMP" not in missing and temp_path.exists():
        fs_temp, temp_raw, _ = read_signal(temp_path)
        sensors["TEMP"] = np.squeeze(temp_raw)
        fs_map["TEMP"] = fs_temp

    fs_acc, acc_raw, _ = read_signal(folder / "ACC.csv")
    acc_mag = np.linalg.norm(acc_raw, axis=1)
    sensors["ACC_MAG"] = acc_mag
    fs_map["ACC_MAG"] = fs_acc
    acc_activity = acc_activity_signal(acc_raw, fs_acc)

    if acc_activity.size:
        sensors["ACC_ACTIVITY"] = acc_activity
        fs_map["ACC_ACTIVITY"] = 1.0 / ACC_ACTIVITY_STEP_SEC


    if "IBI" not in missing:
        sensors["IBI"] = read_ibi(folder / "IBI.csv")
    else:
        sensors["IBI"] = np.array([])
    tags = read_tags(folder / "tags.csv", start_ts)

    if base_id in DUPLICATE_CUTS:
        cuts = DUPLICATE_CUTS[base_id]
        if "EDA" in cuts and "EDA" in sensors:
            sensors["EDA"] = sensors["EDA"][:cuts["EDA"]]
        if "TEMP" in cuts and "TEMP" in sensors:
            sensors["TEMP"] = sensors["TEMP"][:cuts["TEMP"]]
        if "ACC" in cuts and "ACC_MAG" in sensors:
            sensors["ACC_MAG"] = sensors["ACC_MAG"][:cuts["ACC"]]

    duration = len(sensors["EDA"]) / fs_map["EDA"]
    return {
        "sensors": sensors,
        "fs": fs_map,
        "tags": tags,
        "duration": duration,
    }


In [9]:
def subject_baselines(df: pd.DataFrame) -> pd.DataFrame:
    baseline = df[df["label"] == "no_stress"].groupby("subject").mean(numeric_only=True)
    return baseline


## Build windowed dataset

In [10]:
def build_windows_for_subject(state: str, subject: str, tgt_fs: float = 4.0) -> List[dict]:
    info = load_subject_state(state, subject)
    sensors = info["sensors"]
    fs_map = info["fs"]
    tags = info["tags"]
    duration = info["duration"]

    # Resample signals
    eda = resample_to_rate(sensors["EDA"], fs_map["EDA"], tgt_fs)
    temp = resample_to_rate(sensors["TEMP"], fs_map.get("TEMP", tgt_fs), tgt_fs) if "TEMP" in sensors else np.array([])
    acc = resample_to_rate(sensors["ACC_MAG"], fs_map["ACC_MAG"], tgt_fs)
    acc_activity = resample_to_rate(sensors["ACC_ACTIVITY"], fs_map.get("ACC_ACTIVITY", tgt_fs), tgt_fs) if "ACC_ACTIVITY" in sensors else np.array([])

    intervals = make_label_intervals(state, subject, tags, duration)
    windows = window_intervals(duration, WINDOW_SECONDS, WINDOW_STEP_SECONDS)
    rows = []
    for w in windows:
        lbl, span_meta = assign_label(w, intervals)
        if lbl is None or span_meta is None:
            continue
        start_idx = int(w[0] * tgt_fs)
        end_idx = int(w[1] * tgt_fs)
        eda_win = eda[start_idx:end_idx]
        temp_win = temp[start_idx:end_idx] if len(temp) else np.array([])
        acc_win = acc[start_idx:end_idx]
        activity_win = acc_activity[start_idx:end_idx] if len(acc_activity) else np.array([])
        ibi_win = sensors["IBI"][(sensors["IBI"] > 0)] if len(sensors["IBI"]) else np.array([])

        feats = combine_features(eda_win, tgt_fs, acc_win, tgt_fs, temp_win, tgt_fs, ibi_win, activity_win)
        stress_stage = span_meta.get("stage") if isinstance(span_meta, dict) else None
        stress_level = span_meta.get("stress_level") if isinstance(span_meta, dict) else None
        phase_start = span_meta.get("start") if isinstance(span_meta, dict) else None
        phase_end = span_meta.get("end") if isinstance(span_meta, dict) else None
        if lbl == "stress":
            if stress_level is None or np.isnan(stress_level):
                continue
        else:
            stress_level = 0.0 if stress_level is None or np.isnan(stress_level) else stress_level
        phase_label = stress_stage if stress_stage else lbl
        stress_class = stress_bucket(stress_level, phase_label)
        phase_duration = None
        phase_progress = None
        phase_elapsed = None
        if phase_start is not None and phase_end is not None and phase_end > phase_start:
            phase_duration = float(phase_end - phase_start)
            phase_elapsed = float(w[0] - phase_start)
            phase_progress = max(0.0, min(1.0, phase_elapsed / phase_duration))
        row = {
            "subject": base_subject_id(subject),
            "state": state.lower(),
            "phase": phase_label,
            "stress_stage": stress_stage if lbl == "stress" else None,
            "stress_level": float(stress_level),
            "label": stress_class,
            "is_stress": 1 if phase_label in STRESS_PHASES else 0,
            "win_start": w[0],
            "win_end": w[1],
            "phase_start": phase_start,
            "phase_end": phase_end,
            "phase_duration": phase_duration,
            "phase_elapsed": phase_elapsed,
            "phase_progress": phase_progress,
        }
        row.update(feats)
        rows.append(row)
    return rows

def build_dataset(states: List[str] = STATES, max_subjects: int | None = None) -> pd.DataFrame:
    rows = []
    for state in states:
        subjects = sorted([p.name for p in (DATASET_ROOT / state).iterdir() if p.is_dir()])
        if max_subjects:
            subjects = subjects[:max_subjects]
        for subj in subjects:
            try:
                rows.extend(build_windows_for_subject(state, subj))
            except Exception as e:
                print(f"Skip {state}/{subj}: {e}")
                continue
    return pd.DataFrame(rows)

## Build Dataset

In [12]:
df = build_dataset()
df = df.sort_values(["subject", "win_start"]).reset_index(drop=True)

phase_group = df.groupby(["subject", "phase"], dropna=False)
phase_min_start = phase_group["win_start"].transform("min")
phase_max_end = phase_group["win_end"].transform("max")
phase_duration_fallback = (phase_max_end - phase_min_start).replace(0, np.nan)
df["phase_elapsed"] = df["phase_elapsed"].fillna(df["win_start"] - phase_min_start)
df["phase_duration"] = df["phase_duration"].fillna(phase_duration_fallback)
df["phase_progress"] = df["phase_progress"].fillna(
    df["phase_elapsed"] / df["phase_duration"].replace(0, np.nan)
)
df["phase_progress"] = df["phase_progress"].clip(0.0, 1.0).fillna(0.0)
df["phase_position"] = phase_group.cumcount()
phase_counts = phase_group.size().rename("phase_size")
df = df.join(phase_counts, on=["subject", "phase"])
df["phase_position"] = np.where(
    df["phase_size"] > 1,
    df["phase_position"] / (df["phase_size"] - 1),
    0.0,
)
df.drop(columns=["phase_size"], inplace=True)

context_cols = []
for feat in TEMPORAL_FEATURES:
    prev_col = f"prev_{feat}"
    delta_col = f"delta_{feat}"
    roll_mean_col = f"roll_mean_{feat}"
    roll_std_col = f"roll_std_{feat}"
    ema_col = f"ema_{feat}"
    phase_delta_col = f"phase_delta_{feat}"
    phase_z_col = f"phase_z_{feat}"
    subj_group = df.groupby("subject")[feat]
    df[prev_col] = subj_group.shift(1)
    df[delta_col] = df[feat] - df[prev_col]
    roll_mean = (
        subj_group.rolling(window=3, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )
    roll_std = (
        subj_group.rolling(window=3, min_periods=1)
        .std()
        .reset_index(level=0, drop=True)
    )
    df[roll_mean_col] = roll_mean.to_numpy()
    df[roll_std_col] = roll_std.to_numpy()
    df[ema_col] = df.groupby("subject")[feat].transform(lambda s: s.ewm(span=EMA_SPAN, adjust=False).mean())
    phase_first = (
        df.sort_values("win_start")
        .groupby(["subject", "phase"], dropna=False)[feat]
        .transform("first")
    )
    df[phase_delta_col] = df[feat] - phase_first
    phase_running_mean = phase_group[feat].transform(lambda s: s.expanding().mean())
    phase_running_std = phase_group[feat].transform(lambda s: s.expanding().std()).replace(0, np.nan)
    df[phase_z_col] = (df[feat] - phase_running_mean) / phase_running_std
    df[phase_z_col] = df[phase_z_col].replace([np.inf, -np.inf], 0)
    context_cols.extend([prev_col, delta_col, roll_mean_col, roll_std_col, ema_col, phase_delta_col, phase_z_col])

df[context_cols] = df[context_cols].fillna(0)

df.to_csv("stress_level_dataset.csv", index=False)
print("Rows", len(df))
print("Phase counts", df["phase"].value_counts())
print("Label distribution", df["label"].value_counts())
print("Stress level summary", df["stress_level"].describe())

Rows 6742
Phase counts phase
rest                2981
aerobic             1546
anaerobic            998
TMCT                 633
Opposite Opinion     432
Stroop               152
Name: count, dtype: int64
Label distribution label
no_stress          5525
moderate_stress     607
high_stress         355
low_stress          255
Name: count, dtype: int64
Stress level summary count    6742.000000
mean        0.929398
std         2.179948
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        10.000000
Name: stress_level, dtype: float64


In [13]:
# Prepare features for modeling
numeric_cols = [
    c for c in df.columns if c not in FEATURE_EXCLUDE_COLS and np.issubdtype(df[c].dtype, np.number)
]
helper_cols = ["label", "subject", "phase", "win_start"]
train_df = df.loc[:, numeric_cols + helper_cols].copy()
train_df = train_df.dropna(subset=["label"]).reset_index(drop=True)


def balance_by_phase_label(data: pd.DataFrame) -> pd.DataFrame:
    balanced = []
    for phase_name, phase_df in data.groupby("phase"):
        if phase_df.empty:
            continue
        stress_df = phase_df[phase_df["label"] != "no_stress"]
        phase_parts = []
        if not stress_df.empty:
            target = stress_df["label"].value_counts().max()
            target = max(1, int(np.ceil(target * PHASE_WEIGHT_MAP.get(phase_name, 1.0))))
            for label_name, label_df in stress_df.groupby("label"):
                if len(label_df) < target:
                    phase_parts.append(label_df.sample(target, replace=True, random_state=42))
                else:
                    phase_parts.append(label_df)
        else:
            target = len(phase_df)
        no_stress_df = phase_df[phase_df["label"] == "no_stress"]
        if not no_stress_df.empty:
            cap = int(np.ceil(PHASE_BALANCE_NO_STRESS_RATIO * target)) if target else len(no_stress_df)
            if cap > 0:
                if len(no_stress_df) > cap:
                    phase_parts.append(no_stress_df.sample(cap, random_state=42, replace=False))
                else:
                    phase_parts.append(no_stress_df)
        if phase_parts:
            balanced.append(pd.concat(phase_parts, ignore_index=True))
        else:
            balanced.append(phase_df)
    return pd.concat(balanced, ignore_index=True)

train_df = balance_by_phase_label(train_df)
phase_series = train_df["phase"].fillna("unknown")

# Phase-specific baselines (relative to first window in each phase)
phase_sorted = train_df.sort_values(["subject", "phase", "win_start"])
phase_baselines = (
    phase_sorted.groupby(["subject", "phase"], dropna=False)[numeric_cols]
    .transform("first")
)
train_df[numeric_cols] = train_df[numeric_cols] - phase_baselines.fillna(0)

# Subject baselines (computed on retained windows)
baselines = (
    train_df[train_df["label"] == "no_stress"]
    .groupby("subject")[numeric_cols]
    .mean()
    .add_prefix("baseline_")
)
train_df = train_df.join(baselines, on="subject")
baseline_cols = [c for c in train_df.columns if c.startswith("baseline_")]
for col in numeric_cols:
    base_col = f"baseline_{col}"
    if base_col in train_df:
        train_df[col] = train_df[col] - train_df[base_col].fillna(0)
train_df = train_df.drop(columns=baseline_cols, errors="ignore")
train_df = train_df.fillna(0)

feature_cols = numeric_cols
X = train_df[feature_cols]
y = train_df["label"].astype(str)
groups = train_df["subject"].to_numpy()
phases = phase_series.to_numpy()

# Drop columns with all NaNs (after normalization)
non_nan_cols = X.columns[~X.isna().all()]
X = X[non_nan_cols]
feature_cols = non_nan_cols.tolist()

# Scale & encode
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

class_counts = pd.Series(y).value_counts()
class_weight_map = {
    label: len(y) / (len(class_counts) * count)
    for label, count in class_counts.items()
}
class_weight_encoded = {idx: class_weight_map[label] for idx, label in enumerate(le.classes_)}

## Train/evaluate XGBoost
- Grouped by subject to avoid leakage
- Binary target `is_stress`
- Basic hyperparameters; tune as needed

In [14]:
# Grid search over SMOTE k_neighbors
k_values = list(range(1, 11))
results = []

gkf = GroupKFold(n_splits=3 if len(np.unique(groups)) >= 3 else len(np.unique(groups)))

for k in k_values:
    print(f"===== SMOTE k_neighbors={k} =====")
    metrics = []
    cm_accum = np.zeros((len(le.classes_), len(le.classes_)), dtype=float)
    smote = SMOTE(random_state=42, k_neighbors=k)

    for fold, (train_idx, test_idx) in enumerate(gkf.split(X_scaled, y_encoded, groups)):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]
        X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)
        sample_weight = np.array([class_weight_encoded[val] for val in y_train_bal])

        model = XGBClassifier(
            n_estimators=400,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.8,
            colsample_bytree=0.8,
            n_jobs=4,
            objective="multi:softprob",
            num_class=len(le.classes_),
        )
        model.fit(X_train_bal, y_train_bal, sample_weight=sample_weight)
        preds_encoded = model.predict(X_test)
        preds = le.inverse_transform(preds_encoded)
        true_labels = le.inverse_transform(y_test)
        acc = np.mean(preds == true_labels)
        report_dict = classification_report(
            true_labels,
            preds,
            labels=le.classes_,
            output_dict=True,
            zero_division=0,
        )
        cm = confusion_matrix(true_labels, preds, labels=le.classes_)
        cm_accum += cm
        metrics.append(
            {
                "fold": fold,
                "accuracy": acc,
                "macro_f1": report_dict["macro avg"]["f1-score"],
                "weighted_f1": report_dict["weighted avg"]["f1-score"],
            }
        )
    metrics_df = pd.DataFrame(metrics)
    mean_metrics = metrics_df[["accuracy", "macro_f1", "weighted_f1"]].mean()
    avg_cm = cm_accum / max(1, len(metrics))
    results.append(
        {
            "k": k,
            "metrics": metrics_df,
            "mean_metrics": mean_metrics,
            "avg_cm": avg_cm,
        }
    )
    print("Mean metrics:")
    print(mean_metrics)
    print("Average confusion matrix (rows=true, cols=pred):")
    avg_cm_df = pd.DataFrame(
        avg_cm,
        index=[f"true_{c}" for c in le.classes_],
        columns=[f"pred_{c}" for c in le.classes_],
    )
    print(avg_cm_df.round(1))

best_result = max(results, key=lambda r: r["mean_metrics"]["macro_f1"])
best_k_value = best_result["k"]
best_mean_metrics = best_result["mean_metrics"]
best_avg_cm = best_result["avg_cm"]
print("Best SMOTE k_neighbors:", best_k_value)
print("Mean metrics for best k:")
print(best_mean_metrics)
print("Average confusion matrix for best k:")
best_cm_df = pd.DataFrame(
    best_avg_cm,
    index=[f"true_{c}" for c in le.classes_],
    columns=[f"pred_{c}" for c in le.classes_],
)
print(best_cm_df.round(1))
print("Label distribution:")
print(pd.Series(y).value_counts())

===== SMOTE k_neighbors=1 =====
Mean metrics:
accuracy       0.767991
macro_f1       0.489172
weighted_f1    0.744656
dtype: float64
Average confusion matrix (rows=true, cols=pred):
                      pred_high_stress  pred_low_stress  pred_moderate_stress  \
true_high_stress                 119.0             54.3                  78.3   
true_low_stress                    6.0             33.3                 175.7   
true_moderate_stress              26.0             42.3                 122.0   
true_no_stress                     4.3              6.7                  12.7   

                      pred_no_stress  
true_high_stress                42.3  
true_low_stress                 79.0  
true_moderate_stress           103.7  
true_no_stress                1818.0  
===== SMOTE k_neighbors=2 =====
Mean metrics:
accuracy       0.769215
macro_f1       0.497152
weighted_f1    0.750094
dtype: float64
Average confusion matrix (rows=true, cols=pred):
                      pred_high_str

In [15]:
# XGBoost hyperparameter grid search (using best SMOTE k)
if "best_k_value" not in globals():
    raise RuntimeError("Run the SMOTE k grid search cell first.")

param_grid = []
for max_depth in [3, 4, 5]:
    for learning_rate in [0.03, 0.05, 0.1]:
        for subsample in [0.7, 0.85]:
            for colsample in [0.7, 0.9]:
                param_grid.append(
                    {
                        "max_depth": max_depth,
                        "learning_rate": learning_rate,
                        "subsample": subsample,
                        "colsample_bytree": colsample,
                        "n_estimators": 400,
                    }
                )

print(f"Testing {len(param_grid)} parameter combinations")
best_xgb_params = None
best_param_metrics = None
best_param_score = -np.inf
param_results = []
param_gkf = GroupKFold(n_splits=3 if len(np.unique(groups)) >= 3 else len(np.unique(groups)))
smote_for_params = SMOTE(random_state=42, k_neighbors=best_k_value)

for params in param_grid:
    metrics = []
    for fold, (train_idx, test_idx) in enumerate(param_gkf.split(X_scaled, y_encoded, groups)):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]
        X_train_bal, y_train_bal = smote_for_params.fit_resample(X_train, y_train)
        sample_weight = np.array([class_weight_encoded[val] for val in y_train_bal])

        model = XGBClassifier(
            n_estimators=params["n_estimators"],
            learning_rate=params["learning_rate"],
            max_depth=params["max_depth"],
            subsample=params["subsample"],
            colsample_bytree=params["colsample_bytree"],
            n_jobs=4,
            objective="multi:softprob",
            num_class=len(le.classes_),
        )
        model.fit(X_train_bal, y_train_bal, sample_weight=sample_weight)
        preds_encoded = model.predict(X_test)
        preds = le.inverse_transform(preds_encoded)
        true_labels = le.inverse_transform(y_test)
        acc = np.mean(preds == true_labels)
        report_dict = classification_report(
            true_labels,
            preds,
            labels=le.classes_,
            output_dict=True,
            zero_division=0,
        )
        metrics.append(
            {
                "fold": fold,
                "accuracy": acc,
                "macro_f1": report_dict["macro avg"]["f1-score"],
                "weighted_f1": report_dict["weighted avg"]["f1-score"],
            }
        )
    metrics_df = pd.DataFrame(metrics)
    mean_metrics = metrics_df[["accuracy", "macro_f1", "weighted_f1"]].mean()
    param_results.append({"params": params, "metrics": metrics_df, "mean": mean_metrics})
    print(f"Params {params} -> {mean_metrics.to_dict()}")
    if mean_metrics["macro_f1"] > best_param_score:
        best_param_score = mean_metrics["macro_f1"]
        best_xgb_params = params
        best_param_metrics = mean_metrics

print("Best XGB params:", best_xgb_params)
print("Metrics for best params:")
print(best_param_metrics)

Testing 36 parameter combinations
Params {'max_depth': 3, 'learning_rate': 0.03, 'subsample': 0.7, 'colsample_bytree': 0.7, 'n_estimators': 400} -> {'accuracy': 0.7670835971066143, 'macro_f1': 0.5181901086888114, 'weighted_f1': 0.7672945941077268}
Params {'max_depth': 3, 'learning_rate': 0.03, 'subsample': 0.7, 'colsample_bytree': 0.9, 'n_estimators': 400} -> {'accuracy': 0.7645809487776938, 'macro_f1': 0.5095959895511406, 'weighted_f1': 0.7621371666557503}
Params {'max_depth': 3, 'learning_rate': 0.03, 'subsample': 0.85, 'colsample_bytree': 0.7, 'n_estimators': 400} -> {'accuracy': 0.7700050710192753, 'macro_f1': 0.5260378529813996, 'weighted_f1': 0.7701626115756802}
Params {'max_depth': 3, 'learning_rate': 0.03, 'subsample': 0.85, 'colsample_bytree': 0.9, 'n_estimators': 400} -> {'accuracy': 0.7697722810024948, 'macro_f1': 0.5259233150729922, 'weighted_f1': 0.7680359824359018}
Params {'max_depth': 3, 'learning_rate': 0.05, 'subsample': 0.7, 'colsample_bytree': 0.7, 'n_estimators': 40

In [16]:
# Train final model using best settings
if "best_k_value" not in globals():
    raise RuntimeError("Run the SMOTE k grid search cell first.")

final_params = {
    "n_estimators": 400,
    "learning_rate": 0.05,
    "max_depth": 4,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
}
if "best_xgb_params" in globals() and best_xgb_params:
    final_params.update(best_xgb_params)

final_metrics = []
final_cm = np.zeros((len(le.classes_), len(le.classes_)), dtype=float)
phase_records = []
final_smote = SMOTE(random_state=42, k_neighbors=best_k_value)
final_gkf = GroupKFold(n_splits=3 if len(np.unique(groups)) >= 3 else len(np.unique(groups)))

for fold, (train_idx, test_idx) in enumerate(final_gkf.split(X_scaled, y_encoded, groups)):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]
    X_train_bal, y_train_bal = final_smote.fit_resample(X_train, y_train)
    sample_weight = np.array([class_weight_encoded[val] for val in y_train_bal])

    model = XGBClassifier(
        n_estimators=final_params["n_estimators"],
        learning_rate=final_params["learning_rate"],
        max_depth=final_params["max_depth"],
        subsample=final_params["subsample"],
        colsample_bytree=final_params["colsample_bytree"],
        n_jobs=4,
        objective="multi:softprob",
        num_class=len(le.classes_),
    )
    model.fit(X_train_bal, y_train_bal, sample_weight=sample_weight)
    preds_encoded = model.predict(X_test)
    preds = le.inverse_transform(preds_encoded)
    true_labels = le.inverse_transform(y_test)
    acc = np.mean(preds == true_labels)
    report = classification_report(
        true_labels,
        preds,
        labels=le.classes_,
        output_dict=True,
        zero_division=0,
    )
    cm = confusion_matrix(true_labels, preds, labels=le.classes_)
    final_cm += cm
    phase_records.append(
        pd.DataFrame(
            {
                "phase": phases[test_idx],
                "true": true_labels,
                "pred": preds,
            }
        )
    )
    final_metrics.append(
        {
            "fold": fold,
            "accuracy": acc,
            "macro_f1": report["macro avg"]["f1-score"],
            "weighted_f1": report["weighted avg"]["f1-score"],
        }
    )

final_metrics_df = pd.DataFrame(final_metrics)
print("Final cross-validated metrics:")
print(final_metrics_df)
print("Mean metrics:")
print(final_metrics_df[["accuracy", "macro_f1", "weighted_f1"]].mean())

print("Confusion matrix averaged across folds:")
final_cm_df = pd.DataFrame(
    final_cm / len(final_metrics),
    index=[f"true_{c}" for c in le.classes_],
    columns=[f"pred_{c}" for c in le.classes_],
)
print(final_cm_df.round(1))

if phase_records:
    phase_df = pd.concat(phase_records, ignore_index=True)
    phase_summaries = []
    per_phase_class_rows = []
    for phase_name, group_df in phase_df.groupby("phase"):
        phase_report = classification_report(
            group_df["true"],
            group_df["pred"],
            labels=le.classes_,
            output_dict=True,
            zero_division=0,
        )
        phase_summaries.append(
            {
                "phase": phase_name,
                "support": len(group_df),
                "accuracy": np.mean(group_df["true"] == group_df["pred"]),
                "macro_f1": phase_report["macro avg"]["f1-score"],
                "weighted_f1": phase_report["weighted avg"]["f1-score"],
            }
        )
        for label in le.classes_:
            if label not in phase_report:
                continue
            per_phase_class_rows.append(
                {
                    "phase": phase_name,
                    "label": label,
                    "precision": phase_report[label].get("precision", 0.0),
                    "recall": phase_report[label].get("recall", 0.0),
                    "f1": phase_report[label].get("f1-score", 0.0),
                    "support": phase_report[label].get("support", 0),
                }
            )
    phase_summary_df = pd.DataFrame(phase_summaries).sort_values("phase")
    print("Per-phase diagnostics:")
    print(phase_summary_df)
    if per_phase_class_rows:
        phase_class_df = pd.DataFrame(per_phase_class_rows)
        print("Per-phase class metrics:")
        print(phase_class_df.sort_values(["phase", "label"]))

Final cross-validated metrics:
   fold  accuracy  macro_f1  weighted_f1
0     0  0.846939  0.640164     0.848532
1     1  0.739066  0.497972     0.748806
2     2  0.768413  0.514151     0.729804
Mean metrics:
accuracy       0.784806
macro_f1       0.550762
weighted_f1    0.775714
dtype: float64
Confusion matrix averaged across folds:
                      pred_high_stress  pred_low_stress  pred_moderate_stress  \
true_high_stress                 167.0             29.0                  75.3   
true_low_stress                    8.3             54.0                 190.0   
true_moderate_stress              28.7             62.0                 127.0   
true_no_stress                     6.7             17.0                  27.7   

                      pred_no_stress  
true_high_stress                22.7  
true_low_stress                 41.7  
true_moderate_stress            76.3  
true_no_stress                1790.3  
Per-phase diagnostics:
              phase  support  accuracy  